# Setup

In [1]:
!pip install -q --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install -q --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms

In [2]:
import os
import gc
import re
import sys
import ast
import time
import torch
import random
import subprocess
import transformers

import numpy as np
import pandas as pd

from tqdm import tqdm
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, GenerationConfig, BitsAndBytesConfig, PhrasalConstraint

#torch.backends.cuda.enable_mem_efficient_sdp(False)
#torch.backends.cuda.enable_flash_sdp(False)
#os.environ['TORCH_USE_CUDA_DSA'] = "1"

IS_TEST = False

# Model

In [3]:
model_path = "/kaggle/input/deepseek-math"
torch_dtype = torch.float16

# Define quantization
quantize = False
quantize_type = "8-bit"
quantization_config = None

if quantize:
    if quantize_type == "4-bit":
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,  
            bnb_4bit_quant_type="nf4", 
            bnb_4bit_compute_dtype=torch_dtype,
            bnb_4bit_use_double_quant=True)
        
    if quantize_type == "8-bit":
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    
# Load model configs
config = AutoConfig.from_pretrained(model_path)

# Load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  
    torch_dtype=torch_dtype, 
    config=config,
    quantization_config=quantization_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Generation config fo the model
model.generation_config = GenerationConfig.from_pretrained(model_path)

# Set some generation configs for the model
model.generation_config.do_sample = True
model.generation_config.max_new_tokens = 3600
model.generation_config.remove_invalid_values = True
model.generation_config.top_k = 50
model.generation_config.temperature = 1.25
model.generation_config.pad_token_id = model.generation_config.eos_token_id
model.generation_config.renormalize_logits = True
#model.generation_config.penalty_alpha = 0.95

# Skip python word
bad_words_ids = []
bad_words_ids.append(tokenizer("python", add_special_tokens=False).input_ids)
bad_words_ids.append(tokenizer("Python", add_special_tokens=False).input_ids)
bad_words_ids.append(tokenizer("```python", add_special_tokens=False).input_ids)
bad_words_ids.append(tokenizer("```Python", add_special_tokens=False).input_ids)
bad_words_ids.append(tokenizer("```", add_special_tokens=False).input_ids)

# Data processing functions

## Common functions

In [5]:
def naive_parse(answer):
    out = [] 
    start = False 
    end = False  
    for l in reversed(list(answer)): 
        if l in '0123456789' and not end:  
            start = True  
            out.append(l)
        else:
            if start:  
                end = True  
        
    out = reversed(out)  
    return ''.join(out)


def process_output(output):
    result = output   
    flag = True
    
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)
        
        if not len(result_output):
            result_output = re.findall(r'The answer is: \$(\d+)\$', result)
            
            if not len(result_output):
                result_output = re.findall(r'The answer is: (\d+)', result)
                
                if not len(result_output):
                    result_output = re.findall(r'The answer is: \\[(\d+)\\]', result)
                    
                    if not len(result_output):
                        result_output = re.findall(r'The answer is \$(\d+)\$', result)
                    
                        if not len(result_output):
                            result_output = naive_parse(result)
                            flag = True
                            return result_output, flag
                        
                        else:
                            result_output = result_output[-1]
                        
                    else:
                        result_output = result_output[-1]
                        
                else:
                    result_output = result_output[-1] 
                    
            else:
                result_output = result_output[-1]
                
        else:
            result_output = result_output[-1]

        result_output = int(round(float(eval(result_output))) % 1000)
        flag = True
    
    except Exception as e:
        print("\nNot valid result!")
        result_output = random.randint(0, 999)
        flag = False
        
    return result_output, flag

## Single and Few Shot generation functions

In [6]:
def generation(messages):    
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), bad_words_ids=bad_words_ids)
    output = tokenizer.decode(outputs[0][input_tensor.shape[1] : ], skip_special_tokens=True)    
    answer, correct = process_output(output)
    print(f"\nCurrent predicted answer: {answer}. Is correct: {correct}.\n\n\n")
    return answer, correct


def few_shot_generation(problem, examples, task_prompt, n_shots=2):
    messages = []
    
    keys = list(examples.keys())    
    keys = random.sample(keys, n_shots)
    for key in keys:
        example_problem = examples[key]
        messages.append({"role": "user", "content": example_problem['problem'] + "\n" + task_prompt})
        messages.append({"role": "assistant", "content": "\n" + example_problem['solution']})
        
    messages.append({"role": "user", "content": problem + "\n" + task_prompt})
    answer, correct = generation(messages)
    return answer, correct


def single_shot_generation(problem, task_prompt):
    messages = []
    messages.append({"role": "user", "content": problem + "\n" + task_prompt})
    answer, correct = generation(messages)
    return answer, correct
   

def answer_generation(problem, examples, task_prompts, mode="few", n_shots=2, n_iteration=1, result_strategy="most_common"):
    answers = []
    for _ in range(n_iteration):
        for task_prompt in task_prompts:
            try:
                if mode == "single":
                    answer, correct = single_shot_generation(problem, task_prompt)
                if mode == "few":
                    answer, correct = few_shot_generation(problem, examples, task_prompt, n_shots)
                if mode == "tot":
                    answer, correct = single_shot_generation(problem, task_prompt)
                    
                if correct:
                    answers.append(answer)
                else: 
                    print("Not a valid answer!")

                torch.cuda.empty_cache()
                gc.collect()
            except Exception as e:
                print(f"Generation errors + {e}")
    
    result_answer = None
    if result_strategy == "most_common":
        counter = Counter(answers)
        result_answer = counter.most_common(1)[0][0]
    elif result_strategy == "random":
        result_answer = random.sample(answers, 1)
    elif result_strategy == "mean":
        result_answer = np.mean(answers)
    elif result_strategy == "first":
        result_answer = answers[0]
    else:
        result_answer = answers[-1]
        
    return result_answer

## Temperature tree generation

In [7]:
global_start_time = None

class Node:
    def __init__(self, tokenized_text, probability, is_leaf=False):
        self.tokenized_text = tokenized_text
        self.probability = probability
        self.children = []
        self.is_leaf = is_leaf
        
        
def tree_generation(tokenized_text, temperature=1.0, max_new_tokens=96, max_tokens=2200, offset=40): # 200
    try:
        if tokenized_text.dim() == 1:        
            tokenized_text = tokenized_text.unsqueeze(0)

        outputs = model.generate(
            tokenized_text.to(model.device), 
            bad_words_ids=bad_words_ids, 
            do_sample=True, 
            top_p=5.0,
            max_new_tokens=random.randint(max_new_tokens - offset, max_new_tokens + offset), 
            temperature=temperature,
            return_dict_in_generate=True,
            output_scores=True,
        )
    
        ret = "continue"
        if len(outputs.sequences[0]) >= max_tokens:
            ret = "max_tokens"
            
        #elif outputs.sequences[0][-1].item() == model.generation_config.eos_token_id:
        elif model.generation_config.eos_token_id in outputs.sequences[0]:
            ret = "eos"
            
        transition_scores = model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True)
        probability = torch.exp(transition_scores).mean().item()
        
        torch.cuda.empty_cache()        
        return outputs.sequences[0], probability, ret
    
    except Exception as e:
        print(f"Tree generation error: {e}")
        torch.cuda.empty_cache()   
        return None, None, "None"
        

def build_tree(problem, task_prompt="", temperatures=[0.80, 1.20], low_limit_probability=0.92, skip_prob=0.20, max_time=610, starting_tokens=200, start_with_generation=True): # 256
    global global_start_time
    global_start_time = time.time()
    
    messages = []
    messages.append({"role": "user", "content": problem + "\n" + task_prompt})
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    
    if start_with_generation:
        generated_tokenized_text, probability, ret = tree_generation(input_tensor, max_new_tokens=starting_tokens, temperature=0.90)

        if ret == "continue":
            root = Node(generated_tokenized_text, probability)
        else:
            root = Node(input_tensor, 0.0)
    else:
        root = Node(input_tensor, 0.0)
        
    root = build_tree_recursive(
        root=root, 
        temperatures=temperatures, 
        low_limit_probability=low_limit_probability, 
        skip_prob=skip_prob, 
        max_time=max_time
    )
    return root


def build_tree_recursive(root, temperatures=[0.80, 1.20], low_limit_probability=0.92, skip_prob=0.20, max_time=610):
    global global_start_time
    elapsed_time = time.time() - global_start_time
    
    if elapsed_time >= max_time:
        print("Time limit reached!\n")
        return root
    
    print(f"\nElapsed time: {elapsed_time} seconds!")
    
    if not root.is_leaf:
        tokenized_text_root = root.tokenized_text
        
        random.shuffle(temperatures)
        
        for temperature in temperatures:
            elapsed_time = time.time() - global_start_time
            if elapsed_time >= max_time:
                print("Time limit reached!\n")
                return root
            
            if random.random() < skip_prob:
                print("Skip!\n")
                continue
                                            
            generated_tokenized_text, probability, ret = tree_generation(tokenized_text_root, temperature=temperature)
            
            print(f"Generated tokens: {generated_tokenized_text.shape[0]}, Probability: {probability}, Return value: {ret}!\n")
            
            if ret == "None":
                continue

            children = Node(generated_tokenized_text, probability, ret=="eos")
            
            if ret == "continue" and probability >= low_limit_probability:
                children = build_tree_recursive(
                    root=children, 
                    temperatures=temperatures, 
                    low_limit_probability=low_limit_probability, 
                    skip_prob=skip_prob, 
                    max_time=max_time
                )
                    
            if (ret == "eos") or (ret == "continue" and probability >= low_limit_probability):
                root.children.append(children)
    
    gc.collect()
    
    return root


def find_tree_best_path_recursive(root):
    if root.is_leaf or len(root.children) == 0:
        return root.probability, root.tokenized_text, 1
      
    selected_probability = root.children[0].probability
    selected_tokenized_text = root.children[0].tokenized_text
    selected_n_steps = 1
    
    for children in root.children:
        probability, children_tokenized_text, n_steps = find_tree_best_path_recursive(children)
        if probability/n_steps > selected_probability/selected_n_steps:
            selected_probability = probability
            selected_tokenized_text = children_tokenized_text
            selected_n_steps = n_steps
    
    return root.probability + selected_probability, selected_tokenized_text, selected_n_steps + 1


def find_tree_all_paths_recursive(root, selected_paths=None):
    if selected_paths is None:
        selected_paths = []
    
    if root.is_leaf:        
        selected_paths.append(root)

    if len(root.children) > 0:        
        for children in root.children:
            find_tree_all_paths_recursive(children, selected_paths)
        
    return selected_paths
    
    
def answer_generation_tree(
    problem, 
    task_prompt,
    max_iterations=2, # 3
    temperatures=[0.75, 0.825, 0.90, 1.0], #[0.80, 1.25]
    low_limit_probability=0.88, # 0.9215
    skip_prob=0.05, #0.215
    max_time=610, # 630
    start_with_generation=False,
    result_strategy="most_common"
):
    
    iteration = 1
    while(True):
        print(f"\n----- Iteration {iteration} -----")
        try:
            root = build_tree(
                problem=problem, 
                task_prompt=task_prompt, 
                temperatures=temperatures, 
                low_limit_probability=low_limit_probability, 
                skip_prob=skip_prob,
                max_time=max_time, 
                start_with_generation=start_with_generation
            )
            
            if result_strategy == "best":
                best_probability, best_tokenized_text, _ = find_tree_best_path_recursive(root)
                output = tokenizer.decode(best_tokenized_text, skip_special_tokens=True)
                answer, correct = process_output(output)
                break
                
            elif result_strategy == "most_common":
                selected_nodes = find_tree_all_paths_recursive(root)
                answers = []
                for node in selected_nodes:
                    output = tokenizer.decode(node.tokenized_text, skip_special_tokens=True)
                    answer, correct = process_output(output)
                    if correct:
                        answers.append(answer)
                
                counter = Counter(answers)
                answer = counter.most_common(1)[0][0]
                break
                    
            elif result_strategy == "best_valid":
                selected_nodes = find_tree_all_paths_recursive(root)
                selected_nodes.sort(key=lambda node: node.probability, reverse=True)
                for node in selected_nodes:
                    output = tokenizer.decode(node.tokenized_text, skip_special_tokens=True)
                    answer, correct = process_output(output)
                    if correct:
                        break
        
        except Exception as e:
            print(f"Answer generation error for iteration {iteration}: {e}")
            answer = random.randint(0, 999)
                
        if iteration == max_iterations:
            break
        
        iteration += 1
        low_limit_probability -= 0.075
        max_time -= 250
    
    return answer

# Configurations

In [8]:
# AVAILABLE MODES: tot, single, few, temperature_tree
# n_shots: used only in if mode == few
# n_iteration: to run the generation many times
mode = "temperature_tree"  
n_iteration = 1
n_shots = 2    # 
result_strategy = "most_common" 


examples = {
    "algebra" : {
        "problem"  : "Compute $\\frac{x^8+12x^4+36}{x^4+6}$ when $x=5$.",
        "solution" : "Note that $\\left(x^4+6\\right)^2=x^8+12x^4+36$. So $\\frac{x^8+12x^4+36}{x^4+6}=\\frac{\\left(x^4+6\\right)^2}{x^4+6}=x^4+6$. Our answer is therefore $5^4+6=625+6=\\boxed{631}$."
    },
    
    "Geometry" : {
        "problem"  : "What is the smallest possible perimeter, in units, of a triangle whose side-length measures are consecutive integer values?",
        "solution" : "The smallest such triangle has lengths 1, 2, and 3. However, this triangle doesn't work since the sum of any two side lengths must be greater than the third side length (by the Triangle Inequality). The next smallest triangle has lengths 2, 3, and 4, which works. Thus, the smallest possible perimeter is $2+3+4=\\boxed{9}$ units."
    },
    
    "Intermediate Algebra" : {
        "problem"  : "Given that $a-b=5$ and $a^2+b^2=35$, find $a^3-b^3$.",
        "solution" : "We know that $(a-b)^2=a^2-2ab+b^2$. Therefore, we plug in the given values to get $5^2=35-2ab$. Solving, we get that $ab=5$. We also have the difference of cubes factorization $a^3-b^3=(a-b)(a^2+ab+b^2)$. Plugging in the values given and solving, we get that $a^3-b^3=(5)(35+5)=(5)(40)=\\boxed{200}$."
    },
    
    "Prealgebra" : {
        "problem"  : "What is the greatest integer $x$ for which $\\frac79 > \\frac{x}{13}$?",
        "solution" : "Multiplying both sides of the inequality by $13$, we have $\\frac{91}{9}>x$. The largest integer smaller than $\\frac{91}{9}$ is $\\boxed{10}$."
    },
}


task_prompts = [
    "Please reason step by step, and put your final answer within \\boxed{}.",
    "Given the problem, reason step by step, and compute the answer. The final answer is not an expression but is a positive integer and must be inserted within \\boxed{}.",
    "To solve the problem, show all the steps. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression! Then output the final integer answer within \\boxed{}.",
    "Please show all the steps to reach the final answer which is a positive integer and put it within \\boxed{}.",
    "Solve the puzzle step by step. Remember, a step is only valid if it is a piece that leads to the final correct answer (positive integer). Don't add it otherwise!. Enter your final answer within \\boxed{}.",
    "There is only one valid answer. This is a positive integer and not an expression or a literal and must be inserted within \\boxed{}. List the steps to reach it.",
    "Provide the final answer within \\boxed{}, justifying your answer in detail. The final answer is a positive integer and not an expression or a literal.",
    "Thinking step by step, find an answer and put it within \\boxed{}."
]


tot_prompts = [
    """Imagine three different experts who think step by step to solve the problem.
    Each of the experts propose a step of its reasoning, then share it with the group.
    Experts agree on the correctness and then will go on to the next step.
    Experts put the final answer within \\boxed{}.
    """,
    
    """Imagine three different experts who think step by step to solve the problem.
    The first expert proposes the setps, while the other two in sequence check the correctness of each step, correcting them if necessary.
    Experts put the final answer within \\boxed{}.
    """,
]

tree_task_prompt = "Please reason step by step, and put your final answer within \\boxed{}."

if mode == "tot":
    prompts = tot_prompts
else:
    prompts = task_prompts

# Test

In [9]:
if IS_TEST:
    df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')

    for i in tqdm(range(len(df))):
        print(f"\n\n######################## Question {i + 1} ########################")
        try:
            problem = df['problem'].loc[i]
            real_answer = df['answer'].loc[i]
            
            if mode != "temperature_tree":
                predicted_answer = answer_generation(problem, 
                                                     examples, 
                                                     task_prompts=prompts, 
                                                     mode=mode, 
                                                     n_shots=n_shots, 
                                                     n_iteration=n_iteration,
                                                     result_strategy=result_strategy)
            
            else:
                predicted_answer = answer_generation_tree(problem=problem, task_prompt=tree_task_prompt)
                
            print(f"\nPrecited answer: {predicted_answer}")
            print(f"\nReal answer: {real_answer}")
        
        except Exception as e:
            print(f"Some generation errors: {e}\n\n")

# Submission

In [10]:
if not IS_TEST:
    import aimo

    env = aimo.make_env()
    iter_test = env.iter_test()
    
    for test, sample_submission in iter_test:
        print(f"\n\n######################## New Question ########################")
        try:
            problem = test['problem'].values[0]
            
            if mode != "temperature_tree":
                predicted_answer = answer_generation(problem, 
                                                     examples, 
                                                     task_prompts=prompts, 
                                                     mode=mode, 
                                                     n_shots=n_shots, 
                                                     n_iteration=n_iteration,
                                                     result_strategy=result_strategy)
            else:
                predicted_answer = answer_generation_tree(problem=problem, task_prompt=tree_task_prompt)
        except Exception as e:
            print(f"Some generation errors: {e}\n\n")
            predicted_answer = random.randint(0, 999)
        
        print(test)
        print(predicted_answer)
        
        sample_submission['answer'] = predicted_answer
        env.predict(sample_submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


######################## New Question ########################

----- Iteration 1 -----

Elapsed time: 0.053858280181884766 seconds!


2024-06-26 21:49:26.878382: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 21:49:26.878494: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 21:49:27.042741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generated tokens: 77, Probability: 0.818625807762146, Return value: eos!

Generated tokens: 85, Probability: 0.8369529247283936, Return value: eos!

Generated tokens: 75, Probability: 0.8716273903846741, Return value: eos!

Generated tokens: 67, Probability: 0.8256813883781433, Return value: eos!

       id         problem
0  000aaa  What is $1-1$?
0


######################## New Question ########################

----- Iteration 1 -----

Elapsed time: 0.0006647109985351562 seconds!
Generated tokens: 67, Probability: 0.8676378726959229, Return value: eos!

Generated tokens: 64, Probability: 0.868796706199646, Return value: eos!

Generated tokens: 64, Probability: 0.9029936790466309, Return value: eos!

Generated tokens: 66, Probability: 0.8403667211532593, Return value: eos!

       id               problem
0  111bbb  What is $0\times10$?
0


######################## New Question ########################

----- Iteration 1 -----

Elapsed time: 0.0007126331329345703 seconds!
Generated 